## LANE LINE DETECTION

In [1]:
import cv2
import numpy as np

In [2]:
# Function to preprocess the image
def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    return edges

In [3]:
# Function to detect lane lines
def detect_lane_lines(image):
    # Define region of interest (ROI)
    height, width = image.shape
    roi_vertices = [(0, height), (width / 2, height / 2), (width, height)]
    
    # Create a mask to focus on the ROI
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, [np.array(roi_vertices, np.int32)], 255)
    
    # Apply bitwise AND to get the ROI
    masked_image = cv2.bitwise_and(image, mask)
    
    # Use Hough Line Transform to detect lines
    lines = cv2.HoughLinesP(masked_image, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=50)
    
    return lines

In [4]:
# Function to draw detected lane lines on the original image
def draw_lane_lines(image, lines):
    line_image = np.zeros_like(image)
    
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)
    
    return cv2.addWeighted(image, 0.8, line_image, 1, 1)


In [9]:
# Main function
if __name__ == "__main__":
    # Capture video from a camera or load an image
    cap = cv2.VideoCapture('test_video.mp4')  # Replace with your video file or camera source
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Preprocess the frame
        edges = preprocess_image(frame)
        
        # Detect lane lines
        lines = detect_lane_lines(edges)
        
        # Draw lane lines on the frame
        lane_lines = draw_lane_lines(frame, lines)
        
        # Display the result
        cv2.imshow('Lane Detection', lane_lines)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()